In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append(r'C:\Users\solba\dacon-project') 

from models.precessing import train_preprocess_data, test_preprocess_data

In [2]:
raw_train = pd.read_csv("C:\\Users\\solba\\dacon-project\\data\\raw\\train.csv")
raw_test = pd.read_csv("C:\\Users\\solba\\dacon-project\\data\\raw\\test.csv")

In [4]:
train_df = train_preprocess_data(raw_train)
test_df = test_preprocess_data(raw_test)

C:\Users\solba\dacon-project\models\precessing.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  personal['자격증보유개수'] = personal['certificate_acquisition'].apply(count_certificates)
C:\Users\solba\dacon-project\models\precessing.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  personal_df['job'] = le.fit_transform(personal_df['job'])
C:\Users\solba\dacon-project\models\precessing.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index

In [5]:
train_df

,ID,학교,직업,재등록,자격증보유개수,수강개수,투자시간,수료여부
0,TRAIN_000,22,0,0,1,2,2,0
1,TRAIN_001,1,0,0,0,1,3,0
2,TRAIN_002,27,0,0,1,1,10,0
3,TRAIN_003,1,0,0,2,1,2,1
4,TRAIN_004,16,0,0,1,1,2,0
...,...,...,...,...,...,...,...,...
743,TRAIN_743,13,0,0,2,1,3,0
744,TRAIN_744,43,0,0,2,1,3,1
745,TRAIN_745,11,0,0,0,1,1,1
746,TRAIN_746,30,0,0,0,1,2,1


In [5]:
test_df

,ID,학교,직업,재등록,자격증보유개수,수강개수,투자시간
0,TEST_000,3,0,0,0,1,2
1,TEST_001,45,0,0,0,1,1
2,TEST_002,56,0,0,0,1,5
3,TEST_003,48,0,0,4,2,4
4,TEST_004,0,0,0,0,1,1
...,...,...,...,...,...,...,...
809,TEST_809,30,0,0,0,1,1
810,TEST_810,22,0,0,0,1,3
811,TEST_811,20,0,0,0,1,2
812,TEST_812,15,2,0,1,1,2


In [9]:
import pandas as pd

import numpy as np

from sklearn.ensemble import RandomForestClassifier, VotingClassifier

from xgboost import XGBClassifier

from lightgbm import LGBMClassifier

from catboost import CatBoostClassifier # 추가 설치 필요: pip install catboost

from sklearn.preprocessing import StandardScaler



# 1. 고도화된 전처리 (이전보다 더 단순화하여 일반화 성능 높임)

def final_preprocess(df):

    # 활동성 지수: 노이즈를 줄이기 위해 단순히 합산

    df['활동성'] = df['자격증보유개수'] + df['수강개수']

    # 투자시간의 중요도를 강조하되, 이상치 영향 최소화

    df['효율성'] = df['활동성'] / (df['투자시간'] + 1) 

    

    # 범주형은 그대로 문자열 처리

    df['학교'] = df['학교'].astype(str)

    df['직업'] = df['직업'].astype(str)

    return df



train = final_preprocess(train_df.copy())

test = final_preprocess(test_df.copy())



X_train = train.drop(['ID', '수료여부'], axis=1)

y_train = train['수료여부']

X_test = test.drop(['ID'], axis=1)



# 원-핫 인코딩

X_combined_encoded = pd.get_dummies(pd.concat([X_train, X_test]))

X_train_final = X_combined_encoded[:len(train_df)]

X_test_final = X_combined_encoded[len(train_df):]



# 2. 개별 정예 모델 정의 (가장 안정적인 파라미터 조합)

rf = RandomForestClassifier(n_estimators=1000, max_depth=5, class_weight='balanced', random_state=42)



# CatBoost: 범주형 데이터의 왕, 과적합에 매우 강함

cat = CatBoostClassifier(iterations=500, learning_rate=0.01, depth=4, l2_leaf_reg=5, 

                         verbose=0, random_state=42, auto_class_weights='Balanced')



xgb = XGBClassifier(n_estimators=500, learning_rate=0.005, max_depth=3, scale_pos_weight=2.5, random_state=42)



# 3. 가중치 보팅 (Weighted Voting)

# 성능이 더 안정적인 RF와 CatBoost에 더 높은 가중치를 부여 (2:2:1 비율)

voting_model = VotingClassifier(

    estimators=[('rf', rf), ('cat', cat), ('xgb', xgb)],

    voting='soft',

    weights=[2, 2, 1] 

)



# 4. 학습 및 예측

voting_model.fit(X_train_final, y_train)

proba_1 = voting_model.predict_proba(X_test_final)[:, 1]



# 5. 임계값(Threshold) 미세 조정

# 0.39에서 떨어졌다면, 이번엔 1을 조금 더 보수적으로 잡아서(0.48) 정확도를 높여봅니다.

threshold = 0.48 

final_preds = (proba_1 >= threshold).astype(int)



# 6. 결과 저장

print(f"예측된 수료자(1) 수: {sum(final_preds)}")

submission = pd.DataFrame({'ID': test_df['ID'], '수료여부': final_preds})

submission.to_csv('submission_final_boss.csv', index=False)

예측된 수료자(1) 수: 464
